## Scoring POC Notebook

In [158]:
import java.nio.file.Paths
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.{udf, when, split, explode}

In [3]:
var input_dir = Paths.get(System.getProperty("user.home"), "data", "ot", "extract")
var input_file = input_dir.resolve("evidence.json")
val df = spark.read.json(input_file.toString())

input_dir = /home/eczech/data/ot/extract
input_file = /home/eczech/data/ot/extract/evidence.json
df = [access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]


[access_level: string, disease: struct<biosample: struct<id: string, name: string>, efo_info: struct<efo_id: string, label: string ... 2 more fields> ... 3 more fields> ... 12 more fields]

In [102]:
var dfe = df.select(
        $"target.id".as("target_id"),
        $"private.efo_codes".as("efo_codes"),
        $"disease.id".as("disease_id"),
        $"scores.association_score".as("score"),
        $"sourceID".as("source_id"),
        $"id"
    )
var cols = dfe.columns

dfe = [target_id: string, efo_codes: array<string> ... 4 more fields]
cols = Array(target_id, efo_codes, disease_id, score, source_id, id)


Array(target_id, efo_codes, disease_id, score, source_id, id)

In [59]:
dfe.groupBy("source_id").count().orderBy("count").show()

+------------------+------+
|         source_id| count|
+------------------+------+
|   uniprot_somatic|   284|
|           progeny|   308|
|            sysbio|   408|
|    gene2phenotype|  1586|
|            crispr|  1659|
|           intogen|  2375|
|uniprot_literature|  4553|
|       eva_somatic|  7585|
|          reactome| 10159|
|  genomics_england| 10528|
|           uniprot| 30480|
|    phewas_catalog| 55987|
|cancer_gene_census| 60310|
|        slapenrich| 74570|
|               eva| 96734|
|      gwas_catalog|180984|
|  expression_atlas|204229|
|            chembl|383122|
|         phenodigm|500683|
+------------------+------+



In [163]:
var direct_data_sources = List("expression_atlas")
var dfi = dfe.select("id", "source_id", "disease_id", "efo_codes").withColumn(
    "efo_ids", 
    //when($"source_id".isin(direct_data_sources:_*), split($"disease_id", ":").cast("array<string>"))
    when($"source_id".isin(direct_data_sources:_*), array($"disease_id"))
    .otherwise($"efo_codes")
)
dfi

direct_data_sources = List(expression_atlas)
dfi = [id: string, source_id: string ... 3 more fields]


[id: string, source_id: string ... 3 more fields]

In [166]:
var dfie = dfi.select($"id", $"source_id", $"disease_id", explode($"efo_ids"))
dfie

dfie = [id: string, source_id: string ... 2 more fields]


[id: string, source_id: string ... 2 more fields]

In [168]:
dfie.filter($"source_id".isin(direct_data_sources:_*)).limit(3).union(
    dfie.filter(!$"source_id".isin(direct_data_sources:_*)).limit(3)
).show(10, 50)

+--------------------------------+----------------+-----------+-----------+
|                              id|       source_id| disease_id|        col|
+--------------------------------+----------------+-----------+-----------+
|38c0851b3dba497d71ad645a01f498c3|expression_atlas|EFO_0000384|EFO_0000384|
|899d05b84c11212adf3102b3af1a2690|expression_atlas|EFO_0000384|EFO_0000384|
|a56ee52c9af9d70155016194a74ba536|expression_atlas|EFO_0000384|EFO_0000384|
|0ce48d3c91926a55bc6a6dbf40d62123|          chembl|EFO_0000326|EFO_0000616|
|0ce48d3c91926a55bc6a6dbf40d62123|          chembl|EFO_0000326|EFO_0000618|
|0ce48d3c91926a55bc6a6dbf40d62123|          chembl|EFO_0000326|EFO_0000311|
+--------------------------------+----------------+-----------+-----------+

